In [0]:
##### required
from google.colab import drive
drive.mount('/content/drive/')

In [0]:
##### required
!pip install configparser

In [0]:
##### required
!pip install keras==1.2.2

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.regularizers import l2
from keras.optimizers import SGD ,Adagrad
from scipy.io import loadmat, savemat
from keras.models import model_from_json
import theano.tensor as T
import theano
import csv
import configparser
import collections
import time
import os
import skimage.transform
from skimage import color
import numpy as np
import numpy as numpy
from datetime import datetime
from scipy.spatial.distance import cdist,pdist,squareform
import shutil

seed = 7
numpy.random.seed(seed)

In [0]:
def load_model(json_path):  # Function to load the model
    model = model_from_json(open(json_path).read())
    return model

In [0]:
def conv_dict(dict2):
    i = 0
    dict = {}
    for i in range(len(dict2)):
        if str(i) in dict2:
            if dict2[str(i)].shape == (0, 0):
                dict[str(i)] = dict2[str(i)]
            else:
                weights = dict2[str(i)][0]
                weights2 = []
                for weight in weights:
                    if weight.shape in [(1, x) for x in range(0, 5000)]:
                        weights2.append(weight[0])
                    else:
                        weights2.append(weight)
                dict[str(i)] = weights2
    return dict

def load_weights(model, weight_path):
    dict2 = loadmat(weight_path)
    dict = conv_dict(dict2)
    i = 0
    for layer in model.layers:
        weights = dict[str(i)]
        layer.set_weights(weights)
        i += 1
    return model

In [0]:
def load_dataset_One_Video_Features(Test_Video_Path):
    VideoPath =Test_Video_Path
    f = open(VideoPath, "r")
    words = f.read().split()
    num_feat = len(words) / 4096
    
    count = -1;
    VideoFeatues = []
    for feat in range(0, int(num_feat)):
        feat_row1 = np.float32(words[feat * 4096:feat * 4096 + 4096])
        count = count + 1
        if count == 0:
            VideoFeatues = feat_row1
        if count > 0:
            VideoFeatues = np.vstack((VideoFeatues, feat_row1))
    AllFeatures = VideoFeatues

    return  AllFeatures

In [0]:
AllTest_Video_Path = '/content/drive/My Drive/C3D/C3D-v1.0/examples/c3d_feature_extraction/out'
Results_Path = '/result/'
Model_dir='/content/drive/My Drive/C3D/C3D-v1.0/examples/c3d_feature_extraction/out/'

weights_path = Model_dir + 'weights_L1L2.mat'
model_path = Model_dir + 'model.json'

if not os.path.exists(Results_Path):
       os.makedirs(Results_Path)

with open(AllTest_Video_Path+"/test.txt", 'r') as f1:
    for line in f1:
       All_Test_files.append(line.strip())

with open(AllTest_Video_Path+"/label.txt", 'r') as f1:
      for line in f1:
          Label.append(line.strip())

model=load_model(model_path)
load_weights(model, weights_path)
nVideos=len(All_Test_files)
time_before = datetime.now()

accuracy=0
count=0
for iv in range(nVideos):
    Test_Video_Path = os.path.join(AllTest_Video_Path, All_Test_files[iv])
    inputs=load_dataset_One_Video_Features(Test_Video_Path)
    predictions = model.predict_on_batch(inputs)
    aa=All_Test_files[iv]
    aa=aa[0:-4]
    if(Label[iv]==1 and int(max(predictions))>0): 
      accuracy+=1
    elif (Label[iv]==0 and int(max(predictions))==0):
      accuracy+=1
    count+=1
    
print("Accuracy : " +str(accuracy/count))